This is the code that will be imported into our Raspberry Pi to process and send images to our computer for classification

In [1]:
# Import packages

# reload our external packages (tracker)
import importlib

import numpy as np
import cv2
import os
from skimage import data, filters
import tracker as track

importlib.reload(track)

# Directory to save training images
base_dir = "/Users/williamlee/Documents/Git Repos/Lego-Brick-Sorter/Imaging Pipeline/testImages"
os.chdir(base_dir)
print(os.listdir())

['.DS_Store']


In [3]:
# pieceNum = "3021"
# os.chdir(pieceNum)
# Open Video
cap = cv2.VideoCapture(0)

# Create Tracker Object
tracker = track.EuclideanDistTracker()

# Create BG Subtraction Function
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=False)

# Create Window
cv2.startWindowThread()

# FOR TRAINING:
num_img = 0

while(True):
    # print("in loop")
    # Capture video frame by frame
    ret, frame = cap.read()
    
    
    # 0: Pre-Process Image
    window_factor = 0.5       # Size factor of display window
    x_left_crop = 0        # Left side window crop
    x_right_crop = 1920     # Right side window crop
    y_top_crop = 0          # Top side window crop
    y_bottom_crop = 1080    # Bottom side window crop
    saturation_factor = 1 # Factor to saturate image by
    value_factor = 1.2      # Factor to increase value of image by

    # Crop and adjust saturation of image
    frame = frame[y_top_crop:y_bottom_crop, x_left_crop:x_right_crop]
    adjusted = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV).astype("float32")
    (h, s, v) = cv2.split(adjusted)
    s = s * saturation_factor
    s = np.clip(s,0,255)
    v = v * value_factor
    v = np.clip(v,0,255)
    adjusted = cv2.merge([h,s,v])
    frame = cv2.cvtColor(adjusted.astype("uint8"), cv2.COLOR_HSV2BGR)

    # Apply MOG2 to Frame
    fgmask = fgbg.apply(frame)

    # Adjust size
    frame = cv2.resize(frame, None, fx=window_factor, fy=window_factor, interpolation=cv2.INTER_AREA)
    fgmask = cv2.resize(fgmask, None, fx=window_factor, fy=window_factor, interpolation=cv2.INTER_AREA)

    
    # 1: Object Detection
    min_area = 300 # Min area of contour to detect as piece
    
    # Iterate through discovered contours and add detections
    contours, hierarchy = cv2.findContours(image=fgmask, mode=cv2.RETR_CCOMP, method=cv2.CHAIN_APPROX_SIMPLE)
    detections = []
    image_copy = frame.copy()
    for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            # If contour overlap with border then skip
            if((x == 0) or (y == 0) or (x+w >= frame.shape[1]) or (y+h >= frame.shape[0])):
                continue
            
            area = cv2.contourArea(contour)
            if(area > min_area):
                detections.append([x, y, w, h])
                
                
    # 2: Object Tracking
    min_age = 5 # Min age of object to classify as brick
    interval = 1 # Frequency of images to save after being classified as brick
    
    box_ids = tracker.update(detections)
    for box_id in box_ids:
        x, y, w, h, id, age, uniqueId = box_id
        # If a box is "old enough" box is determined to be a piece --> take pictures
        if age > min_age:
            num_img += 1
            larger = w
            if(h > w):
                larger = h
            cv2.rectangle(image_copy, (x,y), (x+larger, y+larger), (0, 255, 0), 3)
            cv2.putText(image_copy, "PIECE IS HERE" + " time:" + str(age) + "id:" + str(uniqueId),(x,y-10),0,0.5,(255, 0, 0), 2)
            
            # Save bounding box as image every X frames
            roi = frame[y:y+larger, x:x+larger]
            #print(str(num_img) + ".jpg")
            true_age = age - min_age
            if(true_age % interval == 0):
                print(true_age)
                #print("save image")
                #cv2.imwrite(str(uniqueId) + "_" + str(num_img) + ".png", roi)
            
    
    cv2.imshow('Feed with bounding boxes', image_copy)
    cv2.imshow('MOG2 Output', fgmask)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()

1
2
3
4
5
1
2
3
1
1
1
2
3
1
1
1
2
1
1
2
3
1
2
3
4
5
1
1
1
2
3
1
2
3
1
2
1
1
2
1
2
1
2
1
2
3
4
5
6
1
2
3
4
5
6
1
2
3
1
4
2
5
3
4
6
7
5
8
9
10
6
1
2
3
4
5
6
7
8
1
2
3
4
5
1
1
2
1
2
3
4
5
6
7
8
9
10
11
12
13
1
2
3
14
15
16
17
18
19
1
1
2
3
4
5
6
1
7
2
3
8
9
4
5
10
6
11
12
1
13
2
3
4
14
15
16
5
17
18
19
6
20
21
22
23
7
8
24
9
10
11
12
13
14
1
2
1
2
3
3
4
5
6
7
4
8
9
5
1
2
6
3
4
7
5
8
6
1
2
3
4
5
6
1
1
2
3
4
5
6
1
2
3
4
5
6
7
8
9
10
11
1
2
1
3
4
5
6
7
1
2
3
4
5
6
1
7
8
2
3
9
10
11
1
1
2
2
3
3
4
4
5
6
1
2
1
2
3
4
5
6
7
8
9
10
11
1
12
13
14
15
16
17
18
19
20
21
1
22
2
1
2
3
1
2
4
3
4
5
5
6
7
6
8
7
9
8
9
10
11
10
11
12
13
12
13
14
14
15
15
16
17
16
17
18
19
20
21
1
22
23
2
3
24
4
5
25
6
7
26
8
9
10
27
28
11
12
29
13
14
15
16
30
17
18
31
19
20
21
1
2
3
4
5
6
7
8
9
10
11
1
2
1
3
4
1
5
6
2
7
3
4
5
8
9
10
6
7
11
8
9
10
11
12
13
14
15
16
17
18
19
20
21
1
2
3
4
1
2
3
1
1
2
3
4
5
6
1
2
3
4
5
6
7
8
1
2
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
1
2
3
4
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
1
1
1
2
3
4

19
3
2
20
21
3
22
23
24
25
26
27
4
28
29
4
30
31
32
33
34
5
5
1
35
36
2
3
37
38
39
6
40
41
42
43
44
45
46
47
7
48
49
8
9
50
51
1
52
53
10
11
12
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
1
2
3
4
80
81
82
83
5
1
6
7
2
3
8
9
10
11
12
13
14
4
5
15
16
17
18
6
1
7
19
20
21
22
8
9
10
2
11
12
13
14
15
1
3
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
1
2
3
4
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
1
39
1
2
40
41
42
43
44
2
3
4
5
6
7
8
9
10
11
45
12
13
14
15
1
2
3
16
4
5
6
7
8
9
10
11
17
18
12
13
14
15
16
1
2
19
3
4
5
6
7
17
18
19
20
21
8
20
9
10
11
12
1
2
13
3
4
5
6
7
8
14
9
10
11
12
1
2
3
4
5
6
13
7
8
1
9
2
10
3
4
5
6
7
8
11
12
9
10
11
1
12
2
3
4
5
6
7
8
9
10
11
13
12
13
14
15
16
1
17
2
3
4
5
6
7
8
9
10
18
19
20
21
22
23
24
25
1
26
2
3
11
12
13
14
15
16
17
18
4
5
6
7
8
27
28
9
10
11
12
13
1
19
14
15
16
17
18
19
20
2
3
4
29
30
31
32
33
5
6
20
7
21
22
8
23
9
10
11
12
13
14
15
16
17
34
35
36
37
38
18
19
20
21
22
23
24
25
26
27
28
29
30
1
31
32

KeyboardInterrupt: 